In [1]:
MSASL_TRAIN_PATH = "./MS-ASL/MSASL_train.json"
MSASL_VAL_PATH = "./MS-ASL/MSASL_val.json"
MSASL_TEST_PATH = "./MS-ASL/MSASL_test.json"
MSASL_CLASSES_PATH = "./MS-ASL/MSASL_classes.json"

In [2]:
import json
import pandas as pd

with open(MSASL_TRAIN_PATH) as f:
    train_data = json.load(f)

with open(MSASL_VAL_PATH) as f:
    val_data = json.load(f)

with open(MSASL_TEST_PATH) as f:
    test_data = json.load(f)
    
with open(MSASL_CLASSES_PATH) as f:
    class_data = json.load(f)

train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)
class_df = pd.DataFrame(class_data)

class_df.columns = ['label_name']
class_df['label'] = class_df.index

train_df = train_df.merge(class_df, how='left', on='label')
val_df = val_df.merge(class_df, how='left', on='label')
test_df = test_df.merge(class_df, how='left', on='label')

In [3]:
train_df

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,label_name
0,match [light-a-MATCH],match,0.000,0,0,0,83,match light-a-MATCH,830,360.0,30.000,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.05754461884498596, 0.21637457609176636, 1.0...",640.0,NaN,match
1,FAIL,fail,0.000,0,-1,0,74,FAIL,542,360.0,25.000,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.0657794177532196, 0.16717177629470825, 0.93...",480.0,NaN,fail
2,laugh,laugh,0.000,4,26,0,31,SignSchool Laugh with Legs 2,312,360.0,29.970,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.13188594579696655, 0.32334136962890625, 1.0...",640.0,NaN,laugh
3,BOOK,book,0.000,0,-1,0,66,BOOK(3),38,360.0,25.000,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.05569887161254883, 0.25173279643058777, 0.9...",480.0,NaN,book
4,sign-language,sign language,0.000,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360.0,29.970,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.03904399275779724, 0.24198183417320251, 1.0...",640.0,NaN,sign language
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16049,WRITE,write,539.939,32,17,16182,16301,ASL 1 Unit 1 Vocabulary,53,360.0,29.970,543.910,https://www.youtube.com/watch?v=fNg_sJ9f8EI,write,"[0.03217703104019165, 0.3161315321922302, 1.0,...",640.0,NaN,write
16050,hot,hot,0.000,2,42,0,127,Hot,135,360.0,29.657,4.282,https://www.youtube.com/watch?v=MkXUHhsMAns,hot,"[0.00036713480949401855, 0.20649650692939758, ...",640.0,NaN,hot
16051,hi,hi,0.000,0,0,0,47,Hi,379,360.0,29.970,1.568,www.youtube.com/watch?v=rhfJGeMDMzQ,hi,"[0.013705313205718994, 0.17314371466636658, 1....",640.0,NaN,hi
16052,Learn,learn,0.000,135,-1,0,104,ASL Learn,22,360.0,30.000,3.467,https://www.youtube.com/watch?v=n-b2NMAwk28,learn,"[0.18269836902618408, 0.19255371391773224, 1.0...",480.0,NaN,learn


In [1]:
import os
import pandas as pd
from moviepy.video.io.VideoFileClip import VideoFileClip
import yt_dlp
import glob

OUTPUT_DIR = "msasl_clips"
NUMBER_OF_CLIPS = 10

def download_and_clip(row, split):
    video_url = row["url"]
    start = float(row["start_time"])
    end = float(row["end_time"])
    label = row["label_name"]
    filename_base = row["file"].replace(" ", "_")

    label_dir = os.path.join(OUTPUT_DIR, split, label)
    os.makedirs(label_dir, exist_ok=True)

    full_video_path_no_ext = os.path.join(label_dir, f"{filename_base}_full")
    clipped_path = os.path.join(label_dir, f"{filename_base}_clip.mp4")

    if os.path.exists(clipped_path):
        return

    ydl_opts = {
        "format": "bestvideo+bestaudio/best",
        "merge_output_format": "mp4",  
        "outtmpl": full_video_path_no_ext,  
        "quiet": True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
    except Exception as e:
        print(f"Failed to download {video_url}: {e}")
        return

    downloaded_files = glob.glob(full_video_path_no_ext + "*")
    if not downloaded_files:
        print(f"Failed to find downloaded file for {video_url}")
        return
    downloaded_path = downloaded_files[0]

    try:
        with VideoFileClip(downloaded_path) as video:
            clip = video.subclip(start, end)
            clip.write_videofile(clipped_path, codec="libx264", audio=False, verbose=False, logger=None)
    except Exception as e:
        print(f"Failed to clip {downloaded_path}: {e}")
        return

    os.remove(downloaded_path)


splits = {
    "train": train_df,
    "val": val_df,
    "test": test_df
}

for split_name, df in splits.items():
    df = df[["url", "start_time", "end_time", "label", "label_name", "file"]]
    clips_to_process = df.head(NUMBER_OF_CLIPS) if NUMBER_OF_CLIPS else df
    for _, row in clips_to_process.iterrows():
        download_and_clip(row, split_name)

NameError: name 'train_df' is not defined

In [4]:
from IPython.display import Video

Video("./msasl_clips/train/book/BOOK(3)_full.mp4", embed=True, width=640, height=360)

In [28]:
import os
import torch
import torch.nn as nn
import random

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_video


class VideoDataset(Dataset):
    def __init__(self, root_dir, transform=None, frame_rate=30, num_frames=30):
        self.root_dir = root_dir
        self.transform = transform
        self.frame_rate = frame_rate
        self.num_frames = num_frames
        self.classes = sorted(os.listdir(root_dir))
        self.video_paths = []
        
        for label in self.classes:
            label_dir = os.path.join(root_dir, label)
            if os.path.isdir(label_dir):
                for file in os.listdir(label_dir):
                    if file.endswith(".mp4"):
                        self.video_paths.append((os.path.join(label_dir, file), label))
        
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path, label = self.video_paths[idx]
        
        frames, _, _ = read_video(video_path, pts_unit="sec")
        
        if len(frames) > self.num_frames:
            step = len(frames) // self.num_frames
            frames = frames[::step][:self.num_frames]
        
        frames = [Image.fromarray(frame.numpy()) for frame in frames]
        
        if self.transform:
            frames = [self.transform(frame) for frame in frames]
        
        frames = torch.stack(frames)

        frames = frames.view(-1, frames.size(1), frames.size(2), frames.size(3))
        
        label_idx = self.classes.index(label)
        
        return frames, label_idx

In [29]:
video_transforms = transforms.Compose([
    transforms.Resize((128, 128)),  
    transforms.RandomHorizontalFlip(),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

In [1]:
train_dir = "./msasl_clips/train/"

train_dataset = VideoDataset(root_dir=train_dir, transform=video_transforms)

train_loader = DataLoader(
    train_dataset,
    batch_size=16,  
    shuffle=True,
    num_workers=0,  
)

for frames, labels in train_loader:
    print(f"Frames shape: {frames.shape}")
    print(f"Labels: {labels}")
    break

NameError: name 'VideoDataset' is not defined

In [ ]:
val_dir = "./msasl_clips/val/"

val_dataset = videodataset(root_dir=val_dir, transform=video_transforms)

val_loader = dataloader(
    val_dataset,
    batch_size=16,  
    shuffle=true,
    num_workers=0,  
)

for frames, labels in train_loader:
    print(f"frames shape: {frames.shape}")
    print(f"labels: {labels}")
    break

In [ ]:
test_dir = "./msasl_clips/test/"

test_dataset = videodataset(root_dir=test_dir, transform=video_transforms)

test_loader = dataloader(
    test_dataset,
    batch_size=16,  
    shuffle=true,
    num_workers=0,  
)

for frames, labels in train_loader:
    print(f"frames shape: {frames.shape}")
    print(f"labels: {labels}")
    break

In [31]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class ResNetForVideoClassification(nn.Module):
    def __init__(self, num_classes, num_frames=30):
        super(ResNetForVideoClassification, self).__init__()
        self.num_frames = num_frames
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
        
    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(-1, 3, 128, 128)  
        
        features = self.resnet(x)  
        features = features.view(batch_size, self.num_frames, -1)
        features = features.mean(dim=1) 
        
        return features

In [32]:
num_classes = len(train_dataset.classes)  
num_frames = 30 
model = ResNetForVideoClassification(num_classes=num_classes, num_frames=num_frames)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for frames, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(frames)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

/home/segus-faultise/Projects/CAB420_GROUP_A/cab420_GROUP_A_ENV/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/segus-faultise/Projects/CAB420_GROUP_A/cab420_GROUP_A_ENV/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 2.1294, Accuracy: 33.33%
Epoch [2/10], Loss: 0.6285, Accuracy: 100.00%
Epoch [3/10], Loss: 0.1824, Accuracy: 100.00%
Epoch [4/10], Loss: 0.0701, Accuracy: 100.00%
Epoch [5/10], Loss: 0.0362, Accuracy: 100.00%
Epoch [6/10], Loss: 0.0214, Accuracy: 100.00%
Epoch [7/10], Loss: 0.0146, Accuracy: 100.00%
Epoch [8/10], Loss: 0.0110, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0083, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0066, Accuracy: 100.00%
